# 기상청 지상 관측자료 - 일자료(기간 조회)
+ API: 기상청 API 허브 https://apihub.kma.go.kr/
+ 참고자료 : https://leedakyeong.tistory.com/entry/Python-Crawling-API-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EA%B3%BC%EA%B1%B0-%EA%B8%B0%EC%83%81-%EA%B4%80%EC%B8%A1-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0
+ 작성: 임경호

### 1. 데이터를 가져올 날짜 설정

In [1]:
import requests
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

# user-defined modules
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [2]:
# 날씨 데이터를 가져올 기간 지정
start_date = '20110101'
end_date = '20111231'

### 2. API를 활용한 데이터 수집 

In [3]:
# url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20151211&tm2=20151214&stn=108&help=1&authKey=853B8q9uTi6dwfKvbq4uqQ'
url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php'
key = "853B8q9uTi6dwfKvbq4uqQ"      #개인 인증키
region = 143                        # STN(ID) 대구 지역코드
# 컬럼 설정: 해당 API 특성 상 사용할 컬럼명을 미리 지정해줘야 사용가능
col_name =["TM","STN","WS_AVG","WR_DAY","WD_MAX","WS_MAX",
           "WS_MAX_TM", "WD_INS", "WS_INS","WS_INS_TM","TA_AVG","TA_MAX",
           "TA_MAX_TM","TA_MIN","TA_MIN_TM", "TD_AVG", "TS_AVG", "TG_MIN",
           "HM_AVG","HM_MIN","HM_MIN_TM", "PV_AVG", "EV_S", "EV_L", 
           "FG_DUR","PA_AVG","PS_AVG", "PS_MAX", "PS_MAX_TM", "PS_MIN",
           "PS_MIN_TM","CA_TOT","SS_DAY", "SS_DUR", "SS_CMB", "SI_DAY",
           "SI_60M_MAX","SI_60M_MAX_TM","RN_DAY", "RN_D99", "RN_DUR", "RN_60M_MAX",
           "RN_60M_MAX_TM","RN_10M_MAX","RN_10M_MAX_TM", "RN_POW_MAX", "RN_POW_MAX_TM", "SD_NEW",
           "SD_NEW_TM","SD_MAX","SD_MAX_TM", "TE_05", "TE_10", "TE_15",
           "TE_30", "TE_50"
           ]

params ={'tm1' : start_date,     # 시작 날짜
         'tm2' : end_date,       # 끝 날짜
         'stn' : region,    # 지역번호
         'help' : 0,        # 0: 설명이 안나옴, 1: 설명이 나옴 
         'authKey' : key}

response = requests.get(url, params=params, verify=False)

In [4]:
if response.status_code == 200:     # URL GET '200 정상'
    text = response.text
    text = text.split("\n")[5:-2] #앞의 필요없는  5줄 부분과 뒤의 필요없는 2줄 부분을 제거
    
    df_weather = pd.DataFrame(text)[0].str.split(expand=True) #데이터프레임 컬럼으로 변경
    df_weather.columns = col_name

In [ ]:
df_weather

### 3. 데이터 전처리

In [ ]:
#데이터 전처리 
df_weather= df_weather[['TM','TA_AVG','TA_MAX','TA_MIN']] #필요한 부분만 추출>|날짜|평균기온|최고기온|최저기온|
df_weather.columns = ['date_id','ta_avg','ta_max','ta_min'] #컬럼명 변경 > SQL과 똑같이 설정

#타입 변경
df_weather['ta_avg'] = df_weather['ta_avg'].astype(float) #실수형으로 변경
df_weather['ta_max'] = df_weather['ta_max'].astype(float) #실수형으로 변경
df_weather['ta_min'] = df_weather['ta_min'].astype(float) #실수형으로 변경

#기온 편차 컬럼 생성
df_weather['ta_diff'] = df_weather['ta_max']-df_weather['ta_min'] 
df_weather

### 4. 테이블에 데이터 저장 (mdw.weather)

In [8]:
import datetime as dt
dt_now = dt.datetime.now()
dt_now_str = dt_now.strftime('%Y-%m-%d %H:%M:%S')

user_name = 'lkh'

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_weather.itertuples():
    sql = "insert into weather (date_id, ta_avg, ta_max, ta_min, ta_diff, create_datetime, create_id) \
                        values (%s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], dt_now_str, user_name))
conn.commit()
conn.close()